In [ ]:
# %% Cell 1: Gerekli Kütüphaneler ve Ayarlar
import os
import cv2
import random
import joblib
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torch.optim as optim
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from func_timeout import func_timeout, FunctionTimedOut
from skimage.feature import hog

# Cihaz ayarı (GPU varsa GPU, yoksa CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:
class CBAMBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super(CBAMBlock, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        self.fc1 = nn.Conv2d(channels, channels // reduction, kernel_size=1, bias=False)
        self.relu = nn.ReLU()
        self.fc2 = nn.Conv2d(channels // reduction, channels, kernel_size=1, bias=False)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = self.fc2(self.relu(self.fc1(self.avg_pool(x))))
        max_out = self.fc2(self.relu(self.fc1(self.max_pool(x))))
        out = self.sigmoid(avg_out + max_out)
        return x * out


In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(in_channels)
    
    def forward(self, x):
        residual = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += residual
        return F.relu(x)


In [ ]:
class HybridEmotionCNN(nn.Module):
    def __init__(self, num_classes=6):
        super(HybridEmotionCNN, self).__init__()
        
        # --- Convolution + BatchNorm + CBAM ---
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, padding=1)  # **Tek kanal giriş**
        self.bn1 = nn.BatchNorm2d(64)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.cbam1 = CBAMBlock(64)

        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.cbam2 = CBAMBlock(128)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.cbam3 = CBAMBlock(256)

        self.conv4 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(512)
        self.pool4 = nn.MaxPool2d(2, 2)
        self.cbam4 = CBAMBlock(512)

        # --- Residual Blocks ---
        self.residual1 = ResidualBlock(512)
        self.residual2 = ResidualBlock(512)
        self.residual3 = ResidualBlock(512)

        # --- Fully Connected Katmanlar ---
        self.fc1 = nn.Linear(512 * 4 * 4, 2048)  # **Düzleştirme işlemi için uygun boyut**
        self.bn6 = nn.LayerNorm(2048)
        self.fc2 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc4 = nn.Linear(512, num_classes)  # 6 sınıf çıktısı

        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        # --- Convolutional Bloklar ---
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.cbam1(x)

        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.cbam2(x)

        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.cbam3(x)

        x = self.pool4(F.relu(self.bn4(self.conv4(x))))
        x = self.cbam4(x)

        # --- Residual Bloklar ---
        x = self.residual1(x)
        x = self.residual2(x)
        x = self.residual3(x)

        # **HATA DÜZELTİLDİ: Flatten işlemi eksikti**
        x = torch.flatten(x, 1)  # Fully Connected katmanlar için düzleştirme

        # --- Fully Connected Katmanlar ---
        x = F.relu(self.bn6(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)  # Sonuç: 6 sınıfın olasılıkları
        
        return x


In [ ]:
# Aynı model yapısını oluşturun ve cihazı ayarlayın
model = HybridEmotionCNN(num_classes=6).to(device)

In [ ]:
# Optimizer ve Loss fonksiyonu tanımlanması
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [ ]:
import os
import shutil
import random

# Kök klasör
base_dir = "C:/Users/Tuna/Downloads/images"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "validation")
test_dir = os.path.join(base_dir, "test")

# Sınıflar (örneğin: happy, sad, etc.)
classes = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d))]

# Test klasörünü oluştur
for cls in classes:
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

for cls in classes:
    print(f"\n▶️ {cls} sınıfı için örnekler hazırlanıyor...")

    # Tüm resimleri topla
    train_cls_path = os.path.join(train_dir, cls)
    val_cls_path = os.path.join(val_dir, cls)

    all_images = []

    for folder in [train_cls_path, val_cls_path]:
        files = [f for f in os.listdir(folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        all_images += [(os.path.join(folder, f), f) for f in files]

    # Karıştır
    random.shuffle(all_images)

    total_count = len(all_images)
    test_count = int(0.2 * total_count)

    print(f"Toplam: {total_count} -> Test'e ayrılacak: {test_count} görsel")

    # Test için seçilen görselleri yeni klasöre kopyala
    selected_test_images = all_images[:test_count]
    for src_path, filename in selected_test_images:
        dst_path = os.path.join(test_dir, cls, filename)
        shutil.copy2(src_path, dst_path)

    print(f"✅ {cls}: Test seti tamamlandı. ({len(selected_test_images)} resim)")

print("\n🎉 Tüm sınıflar için test seti başarıyla oluşturuldu!")


In [ ]:
# %% Cell 2: CNN Modeli Tanımlaması (DeepCNNFeatureExtractor)
# Bu model, 1 kanallı (grayscale) 64x64 görüntüler için sıfırdan eğitilecek şekilde tasarlanmıştır.
class DeepCNNFeatureExtractor(nn.Module):
    def __init__(self):
        super(DeepCNNFeatureExtractor, self).__init__()
        # Blok 1: 1 -> 32 kanal
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.bn1   = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1)
        self.bn2   = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)  # 64x64 -> 32x32

        # Blok 2: 32 -> 64 kanal
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.bn3   = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.bn4   = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)  # 32x32 -> 16x16

        # Blok 3: 64 -> 128 kanal
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.bn5   = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.bn6   = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)  # 16x16 -> 8x8

        # Tam bağlantılı katman: Özellik vektörü boyutu 128*8*8 = 8192'den 256'ya indirme
        self.fc = nn.Linear(128 * 8 * 8, 256)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)

        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)

        x = x.view(x.size(0), -1)
        features = self.fc(x)  # 256 boyutlu özellik vektörü
        return features

# Örnek test (isteğe bağlı)
if __name__ == '__main__':
    dummy = torch.randn(1, 1, 64, 64).to(device)
    feat = DeepCNNFeatureExtractor().to(device)(dummy)
    print("CNN özellik vektörü boyutu:", feat.shape)  # Beklenen: [1, 256]


In [ ]:
# %% Cell 3: HOG Özellik Çıkarım Fonksiyonu
def extract_hog_features(image, pixels_per_cell=(8,8), cells_per_block=(2,2), orientations=9):
    """
    image: Giriş görüntüsü (grayscale, numpy array, boyut: 64x64)
    HOG özellikleri çıkarılır.
    """
    # image'ı float formatına çeviriyoruz
    image = np.float32(image) / 255.0
    features = hog(image, orientations=orientations, pixels_per_cell=pixels_per_cell,
                   cells_per_block=cells_per_block, block_norm='L2-Hys', feature_vector=True)
    return features

# Örnek test
test_image = np.random.randint(0, 256, (64, 64), dtype=np.uint8)
hog_feat = extract_hog_features(test_image)
print("HOG özelliği boyutu:", hog_feat.shape)


In [ ]:
# %% Cell 4: LPG Özellik Çıkarım Fonksiyonu (Sobel Gradyanları ile)
def extract_lpg_features(image):
    """
    image: Giriş görüntüsü (grayscale, numpy array, boyut: 64x64)
    Sobel gradyanları kullanılarak yön histogramı hesaplanır.
    """
    grad_x = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    angle = cv2.phase(grad_x, grad_y, angleInDegrees=True)
    hist = cv2.calcHist([np.uint8(angle)], [0], None, [36], [0, 360])
    hist = cv2.normalize(hist, hist).flatten()
    return hist

# Örnek test
lpg_feat = extract_lpg_features(test_image)
print("LPG özelliği boyutu:", lpg_feat.shape)


In [ ]:
# %% Cell 5: Hibrit Özellik Çıkarım Fonksiyonu (CNN, HOG, LPG)
def extract_hybrid_features(image_tensor, cnn_model, original_image):
    """
    image_tensor: CNN için ön işlenmiş tensör (1x64x64)
    original_image: Orijinal veya yeniden boyutlandırılmış grayscale görüntü (64x64 numpy array)
    """
    # CNN özellikleri
    cnn_model.eval()
    with torch.no_grad():
        feat = cnn_model(image_tensor.unsqueeze(0).to(device))
        cnn_feat = feat.squeeze(0).cpu().numpy()  # Boyut: [256]

    hog_feat = extract_hog_features(original_image)  # Boyut: örneğin ~1764
    lpg_feat = extract_lpg_features(original_image)  # Boyut: [36]

    # Hibrit özellik vektörü: CNN + HOG + LPG
    hybrid_feat = np.concatenate([cnn_feat, hog_feat, lpg_feat])
    return hybrid_feat

# Örnek test
dummy_tensor = torch.randn(1, 64, 64)  # rastgele örnek (1x64x64)
hybrid_test = extract_hybrid_features(dummy_tensor, DeepCNNFeatureExtractor().to(device), test_image)
print("Hibrit özellik vektörü boyutu:", hybrid_test.shape)


In [ ]:
# %% Cell 6: Dataset Sınıfı ve Transform Ayarları

from torch.utils.data import Dataset
import os
import cv2
from PIL import Image
from torchvision import transforms

class EmotionDatasetFromSamples(Dataset):
    def __init__(self, samples, transform=None):
        """
        samples: [(img_path, label)] formatında örneklerin listesi
        transform: Resim için uygulanacak dönüşümler
        """
        self.samples = samples
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        # Orijinal görüntüyü cv2 ile oku ve yeniden boyutlandır
        original_image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if original_image is None:
            raise ValueError(f"Resim okunamadı: {img_path}")
        original_image = cv2.resize(original_image, (64, 64))
        # PIL ile açıp grayscale'e çevir
        image_pil = Image.open(img_path).convert("L")
        if self.transform:
            image_tensor = self.transform(image_pil)
        else:
            from torchvision.transforms import ToTensor
            image_tensor = ToTensor()(image_pil)
        return image_tensor, original_image, label
        
# Transform: Grayscale, Resize (64x64), ToTensor, Normalize
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [ ]:
# %% Cell 7: Hibrit Özellik Çıkarım Fonksiyonu ve Veri Yollarının Ayarlanması
def extract_hybrid_features_from_dataset(dataset, cnn_model):
    features_list, labels_list = [], []
    cnn_model.eval()
    with torch.no_grad():
        for idx, (img_tensor, original_image, label) in enumerate(dataset):
            feat = extract_hybrid_features(img_tensor, cnn_model, original_image)
            features_list.append(feat)
            labels_list.append(label)
            if idx % 100 == 0:
                print(f"{idx} örnek işlendi...")
    return np.array(features_list), np.array(labels_list)

In [ ]:
import os
import shutil
import random

# Mevcut klasörler
base_dir = "C:/Users/Tuna/Downloads/images"
existing_splits = ["train", "validation", "test"]
temp_pool_dir = "C:/Users/Tuna/Downloads/images_all_temp"  # geçici havuz

# Geçici havuz klasörünü oluştur
os.makedirs(temp_pool_dir, exist_ok=True)

# Tüm verileri geçici havuza kopyala (var olan klasör yapısı korunmadan)
for split in existing_splits:
    split_dir = os.path.join(base_dir, split)
    classes = [d for d in os.listdir(split_dir) if os.path.isdir(os.path.join(split_dir, d))]
    for cls in classes:
        class_path = os.path.join(split_dir, cls)
        images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        # Geçici havuzda her sınıf için klasör oluştur
        temp_class_dir = os.path.join(temp_pool_dir, cls)
        os.makedirs(temp_class_dir, exist_ok=True)
        for img in images:
            src = os.path.join(class_path, img)
            dst = os.path.join(temp_class_dir, img)
            shutil.copy2(src, dst)

print("Tüm veriler geçici havuza kopyalandı:", temp_pool_dir)


In [ ]:
train_folder = "C:\\Users\\Tuna\\Downloads\\images_all_temp"  
all_samples = []

classes = sorted(os.listdir(train_folder))

from sklearn.model_selection import train_test_split

classes = sorted(os.listdir(train_folder))

for label, class_name in enumerate(classes):
    class_path = os.path.join(train_folder, class_name)
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        if os.path.exists(img_path):
            all_samples.append((img_path, label))
        else:
            print("Eksik dosya:", img_path)
            
# Eğitim ve test verilerini ayır (60% eğitim, 40% test)
train_samples, test_samples = train_test_split(
    all_samples, test_size=0.4, stratify=[label for _, label in all_samples], random_state=42
)

# Doğrulama ve test verilerini ayır (20% + 20%)
val_samples, test_samples = train_test_split(
    test_samples, test_size=0.5, stratify=[label for _, label in test_samples], random_state=42
)


# Yeni datasetleri oluştur
train_dataset = EmotionDatasetFromSamples(train_samples, transform=transform)
val_dataset = EmotionDatasetFromSamples(val_samples, transform=transform)
test_dataset = EmotionDatasetFromSamples(test_samples, transform=transform)


# Dataloaders oluştur
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, pin_memory=True)
val_loader   = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0, pin_memory=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0, pin_memory=True)

print("Sınıflar:", classes)

In [ ]:
# Öznitelik Çıkarımı
import time
import joblib
import numpy as np
from func_timeout import func_timeout, FunctionTimedOut
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA

# Parametre aralığını daraltılmış ve n_iter belirlenmiş parametre dağılımı
param_distributions = {
    'C': [0.1, 1],
    'gamma': [0.01, 0.1]
}

# SVM için temel fonksiyon
def base_svm(**kwargs):
    kwargs.setdefault('class_weight', 'balanced')
    return SVC(kernel="rbf", **kwargs)

# Hibrit özellik çıkarımı
print("Hibrit öznitelikler çıkarılıyor (CNN + HOG + LPG)...")

# Burada, tüm verinin yeniden bölünmüş datasetlerini (train, validation, test) kullanıyoruz.
# Bu datasetlerin, tüm veriyi bir havuzda toplayıp 60/20/20 oranında böldüğünüz bölümde oluşturduğunuzu varsayıyoruz.
X_train, y_train = extract_hybrid_features_from_dataset(train_dataset, DeepCNNFeatureExtractor().to(device))
X_val, y_val     = extract_hybrid_features_from_dataset(val_dataset, DeepCNNFeatureExtractor().to(device))
X_test, y_test   = extract_hybrid_features_from_dataset(test_dataset, DeepCNNFeatureExtractor().to(device))

print("Öznitelik çıkarımı tamamlandı.")
print("Train veri boyutu: ", X_train.shape)
print("Validation veri boyutu: ", X_val.shape)
print("Test veri boyutu: ", X_test.shape)

# PCA ile boyut indirgeme: Örneğin, orijinal özellik boyutunu 500'e indiriyoruz
pca = PCA(n_components=500, random_state=42)
X_train_reduced = pca.fit_transform(X_train)
X_val_reduced   = pca.transform(X_val)
X_test_reduced  = pca.transform(X_test)

print("PCA sonrası boyutlar:")
print("Train: ", X_train_reduced.shape)
print("Validation: ", X_val_reduced.shape)
print("Test: ", X_test_reduced.shape)

# PCA nesnesini kaydedelim
joblib.dump(pca, "pca_transformer.pkl")


In [ ]:
# Checkpoint kontrolü ve yüklemesi
checkpoint_path = "checkpoint.pth"

if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    if "epoch" in checkpoint:
        start_epoch = checkpoint["epoch"] + 1
        model.load_state_dict(checkpoint["model_state"])
        optimizer.load_state_dict(checkpoint["optimizer_state"])
        print(f"✅ Eğitime {start_epoch}. epoch'tan devam ediliyor.", flush=True)
    else:
        print("⚠️ Checkpoint dosyasında 'epoch' bilgisi bulunamadı! Eğitim 0'dan başlatılıyor.", flush=True)
        start_epoch = 0
else:
    print("⚠️ Checkpoint bulunamadı! Eğitim 0'dan başlatılıyor.", flush=True)
    start_epoch = 0


In [ ]:
checkpoint = torch.load("cnn_model_weights.pth", map_location=device, weights_only=True)
print(checkpoint.keys())  # Checkpoint içindeki tüm anahtarları yazdır

In [ ]:
from tqdm import tqdm

# Alternatif checkpoint kontrolü (weights dosyası)
checkpoint_weights_path = "cnn_model_weights.pth"
if os.path.exists(checkpoint_weights_path):
    checkpoint = torch.load(checkpoint_weights_path, map_location=device)
    print("Checkpoint anahtarları:", checkpoint.keys(), flush=True)
    if "epoch" in checkpoint:
        start_epoch = checkpoint["epoch"] + 1
        print(f"✅ Eğitime {start_epoch}. epoch'tan devam ediliyor.", flush=True)
    else:
        print("⚠️ Checkpoint dosyasında 'epoch' bilgisi bulunamadı! Eğitim 0'dan başlatılıyor.", flush=True)
        start_epoch = 0
else:
    print("⚠️ Alternatif checkpoint bulunamadı! Eğitim 0'dan başlatılıyor.", flush=True)
    start_epoch = 0

epochs = 10  # Kaç epoch devam edeceğini belirle

for epoch in range(start_epoch, epochs):
    model.train()
    running_loss = 0.0

    # tqdm ile batch ilerlemesini gösteriyoruz
    progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}", ncols=100)
    for images, _, labels in progress_bar:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())


    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Ortalama Loss: {avg_loss:.4f}", flush=True)

    # Her epoch sonunda ağırlıkları ve eğitimin ilerleme durumunu kaydet
    torch.save({
        "epoch": epoch,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict()
    }, "checkpoint2.pth")

print("✅ Eğitim tamamlandı ve model kaydedildi!", flush=True)

In [ ]:
# Model ağırlıklarını .pkl uzantılı dosyaya kaydet
with open("cnn_model_weights.pkl", "wb") as f:
    torch.save(model.state_dict(), f)

print("✅ Model ağırlıkları cnn_model_weights.pkl dosyasına kaydedildi!", flush=True)
